## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos para clustering

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario considerando su uso para los métodos de clusterización.

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

Se parte de los mismos datasets con los que se realizó la integración anterior.

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = []
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


Limpieza de los datos (filtrado a nivel de filas)

* Regla 1: dimensión de exactitud (función valores_errores)

In [5]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


* Regla 2: dimensión de exactitud (función valores_errores)

In [6]:
#Se puede definir una función para aplicar los cálculos
min_duracion_credito=df_creditos['duracion_credito'].min()

def regla_duracion_credito_ingresos(row):
    duracion_credito=row.duracion_credito
    situacion_vivienda=row.situacion_vivienda
    pct_ingreso = row.pct_ingreso
    
    if duracion_credito == min_duracion_credito and pct_ingreso>0.6 and situacion_vivienda != "PROPIA" :
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_duracion_credito = df_creditos.apply(lambda row: regla_duracion_credito_ingresos(row), axis=1).rename("regla_duracion_credito")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_duracion_credito], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_credito
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err,ok
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok,ok


In [7]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[(df_creditos['edad'] < 90) & (df_creditos['antiguedad_empleado'] <= 50)]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[(temp['regla_pct_ingresos'] == 'ok') & (temp['regla_duracion_credito'] == 'ok')]

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 9765


Integración de datos

In [8]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 9765


In [9]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 23


#### Supresión de atributos innecesarios

In [10]:
# Se eliminan las columnas calculadas por las reglas de negocio

col_eliminar_df_integrado = ['id_cliente', 'regla_pct_ingresos', 'regla_duracion_credito']

# Se ejecuta la operación

df_integrado.drop(col_eliminar_df_integrado, inplace=True, axis=1)

#### Transformación de atributos

Observación general del dataset

In [11]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9765 entries, 0 to 9764
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   edad                 9765 non-null   int64  
 1   importe_solicitado   9765 non-null   int64  
 2   duracion_credito     9765 non-null   int64  
 3   antiguedad_empleado  9765 non-null   float64
 4   situacion_vivienda   9765 non-null   object 
 5   ingresos             9765 non-null   int64  
 6   objetivo_credito     9765 non-null   object 
 7   pct_ingreso          9765 non-null   float64
 8   tasa_interes         8878 non-null   float64
 9   estado_credito       9765 non-null   int64  
 10  falta_pago           9765 non-null   object 
 11  antiguedad_cliente   9765 non-null   float64
 12  estado_civil         9765 non-null   object 
 13  estado_cliente       9765 non-null   object 
 14  gastos_ult_12m       9765 non-null   float64
 15  genero               9765 non-null   o

##### Procesamiento de valores nulos

En este caso se van a filtrar (eliminar)

In [12]:
df_filtrado = df_integrado[(df_integrado['tasa_interes'].notnull()) & (df_integrado['antiguedad_empleado'].notnull())]
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8878 entries, 0 to 9764
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   edad                 8878 non-null   int64  
 1   importe_solicitado   8878 non-null   int64  
 2   duracion_credito     8878 non-null   int64  
 3   antiguedad_empleado  8878 non-null   float64
 4   situacion_vivienda   8878 non-null   object 
 5   ingresos             8878 non-null   int64  
 6   objetivo_credito     8878 non-null   object 
 7   pct_ingreso          8878 non-null   float64
 8   tasa_interes         8878 non-null   float64
 9   estado_credito       8878 non-null   int64  
 10  falta_pago           8878 non-null   object 
 11  antiguedad_cliente   8878 non-null   float64
 12  estado_civil         8878 non-null   object 
 13  estado_cliente       8878 non-null   object 
 14  gastos_ult_12m       8878 non-null   float64
 15  genero               8878 non-null   object

##### Procesamiento de atributos nominales

Se realiza una binarización por aquellos métodos que no pueden operar con atributos no numéricos.

In [13]:
data = pd.get_dummies(df_filtrado)
data.head()

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,21,1000,2,5.0,9600,0.10,11.14,0,39.0,1144.0,...,True,False,False,True,False,False,False,True,False,False
1,23,35000,2,4.0,65500,0.53,15.23,1,36.0,1887.0,...,True,False,False,True,False,False,False,False,True,False
2,24,35000,4,8.0,54400,0.55,14.27,1,54.0,1314.0,...,True,False,False,True,True,False,False,False,False,False
3,21,2500,2,2.0,9900,0.25,7.14,1,34.0,1171.0,...,True,False,True,False,False,False,False,True,False,False
4,26,35000,3,8.0,77100,0.45,12.42,1,21.0,816.0,...,True,False,False,True,False,False,False,False,False,True


Se exportan los resultados para poder utilizarlos en las operaciones de clustering

In [14]:
data.to_csv("../../data/final/datos_clusterizacion.csv", sep=';', index=False)

In [15]:
print(f"Cantidad de filas del dataset integrado: {data.shape[0]}")
print(f"Cantidad de columnas del dataset integrado: {data.shape[1]}")

Cantidad de filas del dataset integrado: 8878
Cantidad de columnas del dataset integrado: 39
